In [1]:
import autogluon
from autogluon.tabular import TabularDataset, TabularPredictor

import matplotlib.pyplot as plt
import numpy as np
import torch
torch.cuda.get_device_properties(0)

/home/deadscholar/miniconda3/envs/gluon/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


_CudaDeviceProperties(name='NVIDIA RTX A5500 Laptop GPU', major=8, minor=6, total_memory=16383MB, multi_processor_count=58)

In [2]:
# import h5py
# f = h5py.File('../datasets/halfcheetah_medium_expert-v2.hdf5', 'r')
# print(f.keys())
# print(f['actions'].shape[0])
# print(f['infos'].keys())

Import the csv files.

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path

class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str, default = None
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str], default = None
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str], default = None
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool, default = True
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path=None, problem_types=None, eval_metrics=None, consider_labels_correlation=True, **kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        if (problem_types is not None) and (len(problem_types) != len(labels)):
            raise ValueError("If provided, `problem_types` must have same length as `labels`")
        if (eval_metrics is not None) and (len(eval_metrics) != len(labels)):
            raise ValueError("If provided, `eval_metrics` must have same length as `labels`")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = eval_metrics[i]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

In [14]:
# Because the actions should be in [-1, +1] range, we do arctanh, model it, then tanh.
def data_preprocessor(x):
    return np.arctanh(0.995 * np.clip(x, -1.0, +1.0))

def data_un_preprocessor(x):
    return np.clip(np.tanh(x)/0.995, -1.0, +1.0)

In [15]:
from datetime import datetime
date_str = datetime.strftime(datetime.now(), "%d%m-%H%M%S")
print("date_str =", date_str)
print("type of date_str =", type(date_str))

date_str = 1012-185922
type of date_str = <class 'str'>


In [23]:
env_names = ['hopper', 'halfcheetah', 'walker2d']
policy_names = ['random', 'medium-replay', 'medium', 'medium-expert']

for env_name in env_names:
    for policy_name in policy_names:
        exp_name = "{e}-{p}-v2".format(e=env_name, p=policy_name)
        original_train_data = TabularDataset('../datasets/{name}.csv'.format(name=exp_name))
        train_data = data_preprocessor(original_train_data)

        labels = [x for x in original_train_data.keys() if 'ac' in x]  # which columns to predict based on the others
        problem_types = ['regression'] * len(labels)  # type of each prediction problem (optional)
        eval_metrics = ['mean_squared_error'] * len(labels)  # metrics used to evaluate predictions for each label (optional)
        save_path = 'agModels/{name}'.format(name=exp_name)  # specifies folder to store trained models (optional)

        multi_predictor = MultilabelPredictor(labels=labels,
                                              problem_types=problem_types,
                                              eval_metrics=eval_metrics,
                                              path=save_path)
        train_hours = 2
        train_seconds = int(3600 * train_hours / len(labels))
        multi_predictor.fit(train_data, 
                            time_limit=train_seconds,
                            presets=['medium_quality', 'optimize_for_deployment'],
                            excluded_model_types=['KNN'],
                            verbosity=0)

Fitting TabularPredictor for label: ac0 ...
Fitting TabularPredictor for label: ac1 ...
Fitting TabularPredictor for label: ac2 ...
MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('agModels/hopper-random-v2/')
Fitting TabularPredictor for label: ac0 ...
Fitting TabularPredictor for label: ac1 ...
Fitting TabularPredictor for label: ac2 ...
MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('agModels/hopper-medium-replay-v2/')
Fitting TabularPredictor for label: ac0 ...
Fitting TabularPredictor for label: ac1 ...
Fitting TabularPredictor for label: ac2 ...
MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('agModels/hopper-medium-v2/')
Fitting TabularPredictor for label: ac0 ...
Fitting TabularPredictor for label: ac1 ...
Fitting TabularPredictor for label: ac2 ...
MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('agModels/hopper-medium-expert-v2/')
Fitting TabularPredictor for label: ac0 ...
Fitting Ta

In [6]:
exp_name = "halfcheetah-medium-expert-v2"
original_train_data = TabularDataset('../datasets/{name}.csv'.format(name=exp_name))
# train_data = data_preprocessor(original_train_data).sample(n=100000, random_state=0)
train_data = data_preprocessor(original_train_data)

labels = [x for x in original_train_data.keys() if 'ac' in x]  # which columns to predict based on the others
problem_types = ['regression'] * len(labels)  # type of each prediction problem (optional)
eval_metrics = ['mean_squared_error'] * len(labels)  # metrics used to evaluate predictions for each label (optional)
save_path = 'agModels-{name}'.format(name=exp_name)  # specifies folder to store trained models (optional)

print(labels)
print(problem_types)
print(eval_metrics)
print(save_path)

excluded_model_types = ['KNN']

multi_predictor = MultilabelPredictor(labels=labels,
                                      problem_types=problem_types,
                                      eval_metrics=eval_metrics,
                                      path=save_path)
train_hours = 5.5
train_seconds = int(3600 * train_hours / len(labels))
multi_predictor.fit(train_data, 
                   time_limit=train_seconds,
                    presets=['high_quality', 'optimize_for_deployment'],
                    excluded_model_types=['KNN'],
                    num_gpus=1)

/home/deadscholar/miniconda3/envs/gluon/lib/python3.9/site-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: invalid value encountered in arctanh
  result = func(self.values, **kwargs)
Presets specified: ['high_quality', 'optimize_for_deployment']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20


['ac0', 'ac1', 'ac2', 'ac3', 'ac4', 'ac5']
['regression', 'regression', 'regression', 'regression', 'regression', 'regression']
['mean_squared_error', 'mean_squared_error', 'mean_squared_error', 'mean_squared_error', 'mean_squared_error', 'mean_squared_error']
agModels-halfcheetah-medium-expert-v2
Fitting TabularPredictor for label: ac0 ...


Beginning AutoGluon training ... Time limit = 3300s
AutoGluon will save models to "agModels-halfcheetah-medium-expert-v2/Predictor_ac0/"
AutoGluon Version:  0.6.0
Python Version:     3.9.15
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Nov 2 19:50:29 UTC 2022
Train Data Rows:    2000000
Train Data Columns: 18
Label Column: ac0
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    23041.49 MB
	Train Data (Original)  Memory Usage: 288.0 MB (1.2% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in

[1000]	valid_set's l2: 0.358861
[2000]	valid_set's l2: 0.332977
[3000]	valid_set's l2: 0.321029
[4000]	valid_set's l2: 0.312902
[5000]	valid_set's l2: 0.307167
[6000]	valid_set's l2: 0.302794
[7000]	valid_set's l2: 0.299219
[8000]	valid_set's l2: 0.296328
[9000]	valid_set's l2: 0.293859
[10000]	valid_set's l2: 0.292002


	Training S1F2 with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1


...

[1000]	valid_set's l2: 0.220897
[2000]	valid_set's l2: 0.217533
[3000]	valid_set's l2: 0.215309
[4000]	valid_set's l2: 0.213515


	Ran out of time, early stopping on iteration 4409. Best iteration is:
	[4409]	valid_set's l2: 0.212931
	Training S1F8 with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1


[1000]	valid_set's l2: 0.221273
[2000]	valid_set's l2: 0.217819
[3000]	valid_set's l2: 0.215504
[4000]	valid_set's l2: 0.21376
[5000]	valid_set's l2: 0.212362


	Ran out of time, early stopping on iteration 5612. Best iteration is:
	[5612]	valid_set's l2: 0.211637
	-0.2157	 = Validation score   (-mean_squared_error)
	982.17s	 = Training   runtime
	37.64s	 = Validation runtime
Fitting model: LightGBM_BAG_L2 ... Training model for up to 29.46s of the 29.26s of remaining time.
	Memory not enough to fit LGBModel folds in parallel. Will do sequential fitting instead. 	Consider decreasing folds trained in parallel by passing num_folds_parallel to ag_args_ensemble when calling predictor.fit
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	Training S1F1 with GPU, note that this may negatively impact model quality compared to CPU training.
[LightGBM] [Fatal] GPU Tree Learner was not enabled in this build.
Please recompile with CMake option -DUSE_GPU=1
	Ran out of time, early stopping on iteration 25. Best iteration is:
	[25]	valid_set's l2: 0.307398
	Time limit exceeded... Skipping LightGBM_BAG_L2.
Fitting model:

MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('agModels-halfcheetah-medium-expert-v2/')


In [10]:
MultilabelPredictor.load('agModels/halfcheetah-medium-expert-v2_high-quality_01/')